# Polyhedral Compilation with Caten


This tutorial demonstrates how to use `caten.polyhedral` to construct and transform schedules for high-performance kernels, including Matrix Multiplication optimization and Conv-Pool Fusion.


## 1. Setup


In [1]:
import sys
sys.path.append("../")

import caten.isl as I
import caten.polyhedral as P

print("Caten initialized.")

Caten initialized.


## 2. Schedule Constraction

Conv2D

Pool2D

In [2]:
def create_conv_schedule(N, K_out, H_out, W_out, Cin, KH, KW):
    dom_str = f"{{ S_conv[n, k, h, w, c, kh, kw] : 0<=n<{N} and 0<=k<{K_out} and 0<=h<{H_out} and 0<=w<{W_out} and 0<=c<{Cin} and 0<=kh<{KH} and 0<=kw<{KW} }}"
    with P.domain(dom_str) as conv:
        with P.band("{ S_conv[n, k, h, w, c, kh, kw] -> [n, k, h, w, c, kh, kw] }"):
            P.stmt("Out[n, k, h, w] = Out[n, k, h, w], In[n, c, h, w], W[k, c, kh, kw]")
        with P.band("..."):
            pass
    print(conv)
    return conv.finalize()

def create_pool_schedule(N, K_out, H_pool, W_pool, S_pool, KH_pool, KW_pool):
    # Pool Domain
    dom_str = f"{{ S_pool[n, k, h, w, rh, rw] : 0<=n<{N} and 0<=k<{K_out} and 0<=h<{H_pool} and 0<=w<{W_pool} and 0<=rh<{KH_pool} and 0<=rw<{KW_pool} }}"
    with P.domain(dom_str) as pool:
        with P.band("{ S_pool[n, k, h, w, rh, rw] -> [n, k, h, w, rh, rw] }"):
            P.stmt(f"PoolBuf[n, k, h, w] = PoolBuf[n, k, h, w], Out[n, k, h*{S_pool} + rh, w*{S_pool} + rw]")
    return pool.finalize()

In [3]:
create_conv_schedule(10, 3, 3, 5, 2, 3, 3)

AttributeError: 'NoneType' object has no attribute '__repl__'